In [3]:
import re, copy
import heapq

homes = {'A': 3, 'B': 5, 'C': 7, 'D': 9}
costs = {'A': 1, 'B': 10, 'C': 100, 'D': 1000}

class Burrow(object):
    def __init__(self, burrowinput, cost = 0) -> None:
        self.positions = None
        self.amphipods = None
        self.history = []
        self.cost = cost
        self.init_pos_and_amph(burrowinput)
        self.grid = Burrow.get_grid(burrowinput)

    def init_pos_and_amph(self, burrowinput):
        self.positions = []
        self.amphipods = []
        for y, row in enumerate(burrowinput):
            for x, c in enumerate(row):
                if c in [homes.keys(), '.']:
                    self.positions.append((y, x))
                if c in homes.keys():
                    self.amphipods.append(Amphipod(c, y, x))
                    self.positions.append((y, x))
        self.amphipods = sorted(self.amphipods, key = lambda x: x.type)

    def amphipod_is_at_home(self, amphipod):
        if amphipod.x == homes[amphipod.type]:
            return True
        else:
            return False

    def is_final(self):
        return all([self.amphipod_is_at_home(x) for x in self.amphipods])

    def home_wrongly_occupied(self, amphipod):
        home = homes[amphipod.type]
        return not all([a.type == amphipod.type for a in self.amphipods if a.x == home])

    def occupied_positions(self):
        return [(a.y, a.x) for a in self.amphipods]

    def amphipod_positions(self):
        return frozenset([(a.type, a.y, a.x) for a in self.amphipods])

    def printstring(self):
        grid_ = copy.deepcopy(self.grid)
        #print(self.amphipods)
        for a in self.amphipods:
            grid_[a.y][a.x] = a.type
            #print(a.type)
        return Burrow.grid2string(grid_)

    def suggest_next_moves(self, part2 = False): # Priority list
        moves = []
        occ = self.occupied_positions()
        for i, a in enumerate(self.amphipods):
            for p in self.positions:
                if p in occ:
                    continue
                if p[1] in homes.values() and p[1] != homes[a.type]:
                    continue
                if p[0] == 2 and (p[0]+1, p[1]) not in occ:
                    continue
                if part2 and p[0] == 3 and (p[0]+1, p[1]) not in occ:
                    continue
                if part2 and p[0] == 4 and (p[0]+1, p[1]) not in occ:
                    continue
                hasprio = p[0] > 1
                if a.suggested_move(*p, self, hasprio):
                    moves.append((i, *p, hasprio))
        return moves        

    @staticmethod
    def get_grid(burrowinput):
        grid = burrowinput
        for i in range(len(grid)):
            for j in range(len(grid[i])):
                if grid[i][j] in homes:
                    grid[i][j] = '.'
        return grid

    @staticmethod
    def grid2string(grid):
        result = ''
        for row in grid:
            for col in row:
                result += col
            result += '\n'
        return result.strip()

class Amphipod(object):
    def __init__(self, type, y, x) -> None:
        self.type = type
        self.x = x
        self.y = y
    
    def position(self):
        return (self.y, self.x)

    def home_x(self):
        return homes[self.type]

    def move(self, y, x, Burrow: Burrow):
        Burrow.history.append(Burrow.printstring() + "    " + str(Burrow.cost))
        Burrow.cost += (abs(self.x - x) + abs(self.y - y))*costs[self.type]
        if self.y > 1 and y > 1:
            Burrow.cost += (self.y - 1 + y - 1)*costs[self.type]
        self.x = x
        self.y = y
        if self.y == 2 and (self.y + 1, self.x) not in Burrow.occupied_positions(): # will not happen any more
            self.y += 1
            Burrow.cost += costs[self.type]

    def suggested_move(self, y, x, Burrow: Burrow, prio):
        if self.y == y and self.x == x:
            if prio and False:
                print(122)
            return False # no move 
    
        if (y, x) in Burrow.occupied_positions() or (y, x) not in Burrow.positions:
            if prio and False:
                print(y, x)
                print("2")
                print(Burrow.occupied_positions())
                print(Burrow.positions)
            return False
        if y == 1 and (self.y == 1 or x == self.home_x()):
            #print("3")
            return False # not allowed to move in hallway
        if x in homes.values() and x != self.home_x():
            #print("4")
            return False # other homes are not allowed
        if x == self.home_x() and (Burrow.home_wrongly_occupied(self) or self.x == x):
            #print("5555555555")
            return False
        
        # Blocked paths 1/2
        dir = int((x - self.x)/abs(x - self.x))
        occ = Burrow.occupied_positions()
        if y == 1:
            #print(all([(y_, self.x) not in occ for y_ in range(self.y-1, 0, -1)]))
            #print(all([(y, x_) not in occ for x_ in range(self.x, x + dir, dir)]))
            
            if not (all([(y_, self.x) not in occ for y_ in range(self.y - 1, 0, -1)]) and \
                   all([(y, x_) not in occ for x_ in range(self.x, x + dir, dir)])):
                #print("6")
                return False
        # Blocked paths 2/2
        else:
            path = []
            y_ = self.y
            if self.y != 1:
                for y_ in range(self.y - 1, 0, -1):
                    path.append((y_, self.x))
            for x_ in range(self.x + dir, x + dir, dir):
                path.append((y_, x_))
            for y__ in range(y_, y + 1, 1):
                path.append((y__, x_))
            if any([(y, x) in occ for (y, x) in path]):
                return False
        return True
        

In [22]:
import numpy as np
from collections import deque

input_file = "day23.txt"
with open(input_file, "r") as f:
    input_ = f.read().strip()
input_ = input_.splitlines()
for i, line in enumerate(input_):
    input_[i] = list(line)

cost = 0
min_final_cost = np.infty
b = Burrow(input_, cost)
burrows = deque()
burrows.append(b)
it = 0
known_costs = dict()
while burrows:
    it += 1
    b = burrows.popleft()
    #if it % 5000 == 0:
    #    b.print_()
    if b.cost >= min_final_cost:
        continue

    if b.is_final():
        if b.cost < min_final_cost:
            min_final_cost = min(min_final_cost, b.cost)
            print(b.printstring())
            print("Energy cost:", b.cost)
            print("Minimal:", min_final_cost)
            bout = copy.deepcopy(b)
        #if min_final_cost == 12240:
        #    break
        continue

    positions = b.amphipod_positions()
    if positions in known_costs:
        if b.cost >= known_costs[positions]:
            continue

    known_costs[positions] = b.cost
    
    next_moves = b.suggest_next_moves()
    for next_move in next_moves:
        i, y, x, prio = next_move
        b2 = copy.deepcopy(b)
        b2.amphipods[i].move(y, x, b2)
        if b2.cost < min_final_cost:
            if prio:
                burrows.appendleft(b2)
                #print("Priority queue script marker")
            else:
                burrows.append(b2)

    if it >= 1e6:
        break

#############
#...........#
###A#B#C#D###
  #A#B#C#D#
  #########
Energy cost: 20380
Minimal: 20380
#############
#...........#
###A#B#C#D###
  #A#B#C#D#
  #########
Energy cost: 16380
Minimal: 16380
#############
#...........#
###A#B#C#D###
  #A#B#C#D#
  #########
Energy cost: 12382
Minimal: 12382
#############
#...........#
###A#B#C#D###
  #A#B#C#D#
  #########
Energy cost: 12352
Minimal: 12352
#############
#...........#
###A#B#C#D###
  #A#B#C#D#
  #########
Energy cost: 12350
Minimal: 12350
#############
#...........#
###A#B#C#D###
  #A#B#C#D#
  #########
Energy cost: 12282
Minimal: 12282
#############
#...........#
###A#B#C#D###
  #A#B#C#D#
  #########
Energy cost: 12252
Minimal: 12252
#############
#...........#
###A#B#C#D###
  #A#B#C#D#
  #########
Energy cost: 12250
Minimal: 12250
#############
#...........#
###A#B#C#D###
  #A#B#C#D#
  #########
Energy cost: 12242
Minimal: 12242
#############
#...........#
###A#B#C#D###
  #A#B#C#D#
  #########
Energy cost: 12240
Minimal: 12240


In [3]:
min_final_cost
for k in b.history:
    print(k)

#############
#...........#
###A#D#B#D###
  #B#C#A#C#
  #########    0
#############
#.A.........#
###.#D#B#D###
  #B#C#A#C#
  #########    2
#############
#.A.B.......#
###.#D#.#D###
  #B#C#A#C#
  #########    42
#############
#.A.B......A#
###.#D#.#D###
  #B#C#.#C#
  #########    48
#############
#.A.B.....DA#
###.#D#.#.###
  #B#C#.#C#
  #########    2048
#############
#.A.B.....DA#
###.#D#.#.###
  #B#C#C#.#
  #########    2648
#############
#.A.B......A#
###.#D#.#.###
  #B#C#C#D#
  #########    5648
#############
#.A.B......A#
###.#.#.#D###
  #B#C#C#D#
  #########    11648
#############
#.A.B.C....A#
###.#.#.#D###
  #B#.#C#D#
  #########    11948
#############
#.A.B......A#
###.#.#C#D###
  #B#.#C#D#
  #########    12148
#############
#.A........A#
###.#.#C#D###
  #B#B#C#D#
  #########    12178
#############
#.A.B......A#
###.#.#C#D###
  #.#B#C#D#
  #########    12208
#############
#.A........A#
###.#B#C#D###
  #.#B#C#D#
  #########    12228
#############
#.A.........#
###.#B#C#D###


In [23]:
it

900040

In [24]:
len(burrows)

0

<h1>Part 2</h1>

In [10]:
import numpy as np
from collections import deque

input_file = "day23.txt"
with open(input_file, "r") as f:
    input_ = f.read().strip()
input_ = input_.splitlines()
for i, line in enumerate(input_):
    input_[i] = list(line)
input_.insert(3, list('  #D#C#B#A#'))
input_.insert(4, list('  #D#B#A#C#'))

cost = 0
min_final_cost = np.infty
b = Burrow(input_, cost)
burrows = deque()
burrows.append(b)
it = 0
known_costs = dict()
while burrows:
    it += 1
    b = burrows.popleft()
    if it % 10000 == 0:
        print(b.printstring())

    if b.cost >= min_final_cost:
        continue

    if b.is_final():
        if b.cost < min_final_cost:
            min_final_cost = min(min_final_cost, b.cost)
            print(b.printstring())
            print("Energy cost:", b.cost)
            print("Minimal:", min_final_cost)
            bout = copy.deepcopy(b)
        if min_final_cost == 44618:
            break
        continue

    positions = b.amphipod_positions()
    if positions in known_costs:
        if b.cost >= known_costs[positions]:
            continue

    known_costs[positions] = b.cost
    
    next_moves = b.suggest_next_moves(part2 = True)
    for next_move in next_moves:
        i, y, x, prio = next_move
        b2 = copy.deepcopy(b)
        b2.amphipods[i].move(y, x, b2)
        if b2.cost < min_final_cost:
            if prio:
                burrows.appendleft(b2)
                #print("Priority queue script marker")
            else:
                burrows.append(b2)

    if it >= 1e6:
        break

#############
#B..D.....DA#
###.#.#.#.###
  #D#C#B#A#
  #D#B#A#C#
  #B#C#A#C#
  #########
#############
#B....C.B.D.#
###A#.#.#D###
  #D#.#B#A#
  #D#.#A#C#
  #B#C#A#C#
  #########
#############
#AB.D...D..D#
###.#D#B#.###
  #.#C#B#A#
  #.#B#A#C#
  #.#C#A#C#
  #########
#############
#.D.D.C.D..A#
###.#.#B#D###
  #.#.#B#A#
  #.#B#A#C#
  #B#C#A#C#
  #########
#############
#.B.D.B.C.D.#
###A#.#.#.###
  #D#.#B#A#
  #D#.#A#C#
  #B#C#A#C#
  #########
#############
#DC.....D.AB#
###A#.#B#.###
  #D#.#B#.#
  #D#.#A#C#
  #B#C#A#C#
  #########
#############
#.C.B...C..D#
###A#.#.#D###
  #D#.#B#A#
  #D#.#A#C#
  #B#B#A#C#
  #########
#############
#AD...B.D.DB#
###.#.#.#.###
  #.#C#.#A#
  #D#B#A#C#
  #B#C#A#C#
  #########
#############
#DD.A...D.AB#
###.#D#.#.###
  #.#C#B#.#
  #.#B#A#C#
  #B#C#A#C#
  #########
#############
#DD.D...A.CA#
###.#.#B#.###
  #.#C#B#.#
  #D#B#A#.#
  #B#C#A#C#
  #########
#############
#BD...C.D.AB#
###.#.#.#D###
  #.#.#.#A#
  #D#B#A#C#
  #B#C#A#C#
  #########
##########

In [11]:
min_final_cost

44618

In [12]:
it

1000002

In [7]:
print(b.printstring())

#############
#CA.D.A.B.CD#
###.#.#.#.###
  #B#.#.#.#
  #########


In [13]:
len(burrows)

36538

In [15]:
it

142493